In [3]:
import pandas as pd
import sqlite3
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

# Reddit news

In [ ]:
import string
string.punctuation

In [ ]:
kolonas = ['date']+['top'+str(i) for i in range(1,31)]
print(kolonas)

In [ ]:
df = pd.DataFrame(columns = kolonas)

In [ ]:
connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()

In [ ]:
sql = ("Select date(moment) as date, title from top30 where date = '2021-05-05'")
z = pd.read_sql_query(sql,connection)

In [ ]:
for d in z.date.unique():
    atbilde =[]
    atbilde.append(d)
    #print(atbilde)
    for title in z.title[z.date == d]:
        atbilde.append(title)
        
    df.loc[len(df)] = atbilde

In [ ]:
df.to_sql('main', connection, if_exists='append',index=False)

In [ ]:
atbilde = []
atbilde.append(z.date[0])
#print(atbilde)

for title in z.title:
    atbilde.append(title)
        
print(atbilde)
df.loc[len(df)] = atbilde

# delete

connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()
sql = ("DELETE from top30 where moment  between '2021-03-30 11:17:11.915369' and '2021-03-30 11:18:48.647641'")
cursor.execute(sql)
connection.commit()

# Yahoo finance

In [2]:
from bs4 import BeautifulSoup
import requests
import re

In [ ]:
df = pd.DataFrame(columns = ['date','SP500_Open', 'SP500_High', 'SP500_Low', 'SP500_Close', 'SP500_Adj_Close', 'SP500_Volume'])
df

In [ ]:
str([str('SP500_')+i for i in df.columns])

## S&P 500 (^GSPC)
SNP - SNP Real Time Price. Currency in USD

In [ ]:
url = "https://finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC"

In [ ]:
r = requests.get(url)
data = r.text
soup = BeautifulSoup(data)

In [ ]:
answer = []
for i in soup.find_all('td')[0:7]:
    #print(i.span)
    print(i.span.text)
    answer.append(i.span.text)
    #print(len(answer)    

for i in range(0, int(len(answer))//7):
    atbilde = []
    #print(i)
    for j in range (0,7):
        if j == 0:
            atbilde.append(pd.to_datetime(answer[i*7+j], infer_datetime_format=True))
        else:
            atbilde.append(answer[i*7+j])
    df.loc[len(df)] = atbilde

In [ ]:
connection = sqlite3.connect('MSC/MSC.db')
cursor = connection.cursor()
sql = ("Select date(moment) as date, title from top30 where moment>date('now')")
z = pd.read_sql_query(sql,connection)

atbilde = []
columnnames = "'date', 'top1', 'top2', 'top3', 'top4', 'top5', 'top6', 'top7', 'top8', 'top9', 'top10', 'top11', 'top12', 'top13', 'top14', 'top15', 'top16', 'top17', 'top18', 'top19', 'top20', 'top21', 'top22', 'top23', 'top24', 'top25', 'top26', 'top27', 'top28', 'top29', top30"
insertintotable = '?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?'
atbilde = [z.date[0]] +[i for i in  z.title]

connection = sqlite3.connect('MSC/MSC.db')
cursor = connection.cursor()
sql2 = f'cursor.execute("insert INTO  main ({columnnames}) VALUES ({insertintotable})", {atbilde})'
eval(sql2)
connection.commit()
connection.close()
print("Appended to MAIN")

In [ ]:
connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()
sql = ("Select * from DJI")
z = pd.read_sql_query(sql,connection).columns
z

In [ ]:
connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()
sql = ("Select max(date) as date from SP500")
z = pd.read_sql_query(sql,connection)
latest_date = z.date[0]

##SP500
url = "https://finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC"
r = requests.get(url)
data = r.text
soup = BeautifulSoup(data)

answer = []
if str(pd.to_datetime(soup.find_all('td')[0].text, infer_datetime_format=True)) == latest_date:
    print('Stop. This data point is already in the data base')
else:
    print('Lets get scraping')
    answer.append(str(pd.to_datetime(soup.find_all('td')[0].text, infer_datetime_format=True)))
    answer = answer + [i.span.text for i in soup.find_all('td')[1:7]]
    
    columnnames = "'Date', 'SP500_Open', 'SP500_High', 'SP500_Low', 'SP500_Close','SP500_Adj_Close', 'SP500_Volume'"
    insertintotable = '?,?,?,?,?,?,?'
    
    connection = sqlite3.connect('MSC/MSC.db')
    cursor = connection.cursor()
    sql2 = f'cursor.execute("insert INTO  SP500 ({columnnames}) VALUES ({insertintotable})", {answer})'
    eval(sql2)
    connection.commit()
    connection.close()
    print(f"{answer[0]} appended to SP500")
    
    
    ## DJI
    url = "https://finance.yahoo.com/quote/%5EDJI/history?p=%5EDJI"
    r = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data)
    
    answer = []
    answer.append(str(pd.to_datetime(soup.find_all('td')[0].text, infer_datetime_format=True)))
    answer = answer + [i.span.text for i in soup.find_all('td')[1:7]]
    
    columnnames = "'date', 'SP500_Open', 'SP500_High', 'SP500_Low', 'SP500_Close','SP500_Adj_Close', 'SP500_Volume'"
    insertintotable = '?,?,?,?,?,?,?'
    
    connection = sqlite3.connect('MSC/MSC.db')
    cursor = connection.cursor()
    sql2 = f'cursor.execute("insert INTO  SP500 ({columnnames}) VALUES ({insertintotable})", {answer})'
    eval(sql2)
    connection.commit()
    connection.close()
    print(f"{answer[0]} appended to DJI")
    
    
answer

In [ ]:
df.to_sql('SP500', connection, if_exists='append',index=False)

## Dow Jones Industrial Average (^DJI)
DJI - DJI Real Time Price. Currency in USD

In [ ]:
url = "https://finance.yahoo.com/quote/%5EDJI/history?p=%5EDJI"

In [ ]:
df = pd.DataFrame(columns = ['date', 'DJI_Open', 'DJI_High', 'DJI_Low', 'DJI_Close', 'DJI_Adj_Close', 'DJI_Volume'])
df

In [ ]:
columns_new = [str('DJI_')+i for i in df.columns]

In [ ]:
r = requests.get(url)
data = r.text
soup = BeautifulSoup(data)

In [ ]:
answer = []
for i in soup.find_all('td'):
    #print(i.span)
    #print(i.span.text)
    answer.append(i.span.text)
    #print(len(answer)  

In [ ]:
for i in range(0, int(len(answer))//7):
    atbilde = []
    #print(i)
    for j in range (0,7):
        if j == 0:
            atbilde.append(pd.to_datetime(answer[i*7+j], infer_datetime_format=True))
        else:
            atbilde.append(answer[i*7+j])
    df.loc[len(df)] = atbilde

In [ ]:
df

In [ ]:
df.to_sql('DJI', connection, if_exists='append',index=False)

# SP500 companies

In [2]:
from bs4 import BeautifulSoup
import requests
import re

df = pd.read_excel('SP500.xlsx')

In [4]:
connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()
sql = ("Select * from SP500_companies")
df = pd.read_sql_query(sql,connection)
df.head()

,Company,Symbol,Weight
0,Apple Inc.,AAPL,5.664493
1,Microsoft Corporation,MSFT,5.264839
2,Amazon.com Inc.,AMZN,3.886738
3,Facebook Inc. Class A,FB,2.109076
4,Alphabet Inc. Class A,GOOGL,1.925453


## Wikipedia

In [1]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

In [4]:
r = requests.get(url)
data = r.text
soup = BeautifulSoup(data)

In [9]:
answer = []
for i in soup.find_all('td')[0:7]:
    print(i)


<td><a class="external text" href="https://www.nyse.com/quote/XNYS:MMM" rel="nofollow">MMM</a>
</td>
<td><a href="/wiki/3M" title="3M">3M Company</a></td>
<td><a class="external text" href="https://www.sec.gov/cgi-bin/browse-edgar?CIK=MMM&amp;action=getcompany" rel="nofollow">reports</a></td>
<td>Industrials</td>
<td>Industrial Conglomerates</td>
<td><a class="mw-redirect" href="/wiki/St._Paul,_Minnesota" title="St. Paul, Minnesota">St. Paul, Minnesota</a></td>
<td>1976-08-09</td>


In [28]:
for i in soup.find_all('tr'):
    for j in i.find_all('th'):
        print(j)

<th><a href="/wiki/Ticker_symbol" title="Ticker symbol">Symbol</a>
</th>
<th>Security</th>
<th><a href="/wiki/SEC_filing" title="SEC filing">SEC filings</a></th>
<th><a href="/wiki/Global_Industry_Classification_Standard" title="Global Industry Classification Standard">GICS</a> Sector</th>
<th>GICS Sub-Industry</th>
<th>Headquarters Location</th>
<th>Date first added</th>
<th><a href="/wiki/Central_Index_Key" title="Central Index Key">CIK</a></th>
<th>Founded
</th>
<th data-sort-type="date" rowspan="2">Date
</th>
<th colspan="2">Added
</th>
<th colspan="2">Removed
</th>
<th rowspan="2">Reason
</th>
<th>Ticker</th>
<th>Security</th>
<th>Ticker</th>
<th>Security
</th>


In [40]:
import wikipedia as wp
def get_table(title, filename, match, use_cache=False):

    if use_cache and os.path.isfile(filename):
        pass
    else:
        html = wp.page(title).html()
        df = pd.read_html(html, header=0, match=match)[0]
        
        df.to_csv(filename, header=True, index=False, encoding='utf-8')
            
    df = pd.read_csv(filename)
    return df

In [42]:
#title = 'List of S&P 500 companies'
title = 'Selected changes to the list of S&P 500 components'
filename = 'sp500.csv'
sp500 = get_table(title, filename, match='Symbol')

from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%m/%d/%Y %H:%M:%S")
print('{} (retrieved {})'.format(title, dt_string))
sp500

Selected changes to the list of S&P 500 components (retrieved 05/20/2021 17:53:33)


,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [48]:
#html = 
#wp.page(title).html()
#df = 
pd.read_html(html)[1]

Date  Added                             Removed  \
                 Date Ticker                    Security  Ticker   
0        May 14, 2021    CRL  Charles River Laboratories    FLIR   
1      April 20, 2021    PTC                     PTC Inc     VAR   
2      March 22, 2021   NXPI          NXP Semiconductors     FLS   
3      March 22, 2021   PENN        Penn National Gaming     SLG   
4      March 22, 2021   GNRC            Generac Holdings     XRX   
..                ...    ...                         ...     ...   
266  December 5, 2000    SBL         Symbol Technologies      OI   
267  December 5, 2000    AYE            Allegheny Energy     GRA   
268  December 5, 2000    ABK             Ambac Financial     CCK   
269     July 27, 2000   JDSU                JDS Uniphase     RAD   
270  December 7, 1999   YHOO                      Yahoo!     LDW   

                                                                        Reason  
                   Security                                             Reason  
0              FLIR Systems  S&P 500 constituent Teledyne Technologies Inc....  
1    Varian Medical Systems  Siemens Healthineers AG acquired Varian Medica...  
2                 Flowserve                   Market capitalization change.[8]  
3           SL Green Realty                   Market capitalization change.[8]  
4                     Xerox                   Market capitalization change.[8]  
..                      ...                                                ...  
266          Owens-Illinois                                Market Cap changes.  
267              W.R. Grace                                Market Cap changes.  
268          Crown Holdings                                Market Cap changes.  
269                 RiteAid                            Market Cap change.[212]  
270                 Laidlaw                            Market Cap change.[213]  

[271 rows x 6 columns]

In [37]:
sp500.to_csv("SP500.csv")

### Insert DJI and SP500 in the same table
### One less step in ETL

connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()

columnnames = "'Company', 'Symbol', 'Weight'"
insertintotable = '?,?,?'

answer = ['Dow Jones Industrial Average', '^DJI', '100']
sql2 = f'cursor.execute("insert INTO  SP500_companies ({columnnames}) VALUES ({insertintotable})", {answer})'
eval(sql2)
connection.commit()
connection.close()

In [ ]:
columnnames = "'date', 'open', 'high', 'low', 'close', 'adj_close','volume', 'symbol'"
insertintotable = '?,?,?,?,?,?,?,?'

for i in df.Symbol[490:]:
    r = requests.get(f'https://finance.yahoo.com/quote/{i}/history?p={i}')
    print(f'https://finance.yahoo.com/quote/{i}/history?p={i}')
    data = r.text
    soup = BeautifulSoup(data, features="lxml")
    ofset = 0
    for j in range(0, int(len(soup.find_all('td')))//7):  
        #if type(soup.find_all('td')[j*7+1+ofset].span.text) != int and type(soup.find_all('td')[j*7+1+ofset].span.text) != float:
        if soup.find_all('td')[j*7+1+ofset].span.text == "Dividend" or soup.find_all('td')[j*7+1+ofset].span.text == "Stock Split":
            ofset += 2
        try:
            answer = []
            answer.append(str(pd.to_datetime(soup.find_all('td')[j*7+ofset].text, infer_datetime_format=True)))
            answer = answer + [i.span.text for i in soup.find_all('td')[j*7+1+ofset:j*7+7+ofset]] + [i]
        except:
            answer = []
            ofset += 2
        print(answer)
        connection = sqlite3.connect('MSC.db')
        cursor = connection.cursor()
        sql2 = f'cursor.execute("insert INTO  prices ({columnnames}) VALUES ({insertintotable})", {answer})'
        eval(sql2)
        connection.commit()
        connection.close()    

In [ ]:
#for symbol in df
no_financials = "'BRK.B', 'BF.B', 'FLIR'"

connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()
sql = ('''
Select symbol from SP500_companies 
where symbol not in (select distinct symbol from prices where date == '2021-05-19 00:00:00')
and  symbol not in ('BRK.B', 'BF.B', 'FLIR')'''
    )
df = pd.read_sql_query(sql,connection)

connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()
sql = ("Select * from prices where date ")
z = pd.read_sql_query(sql,connection)
connection.commit()
connection.close()

columnnames = "'date', 'open', 'high', 'low', 'close', 'adj_close','volume', 'symbol'"
insertintotable = '?,?,?,?,?,?,?,?'

for i in df.Symbol[2:]:
    r = requests.get(f'https://finance.yahoo.com/quote/{i}/history?p={i}')
    print(f'https://finance.yahoo.com/quote/{i}/history?p={i}')
    data = r.text
    soup = BeautifulSoup(data, features="lxml")
    counter = 0
    for j in range (0, 4):
        if soup.find_all('td')[1+j*7].span != None and pd.to_datetime(soup.find_all('td')[j*7].text, infer_datetime_format=True) > pd.to_datetime(z.date[z.symbol == i].iloc[0]) and counter == 0:
            try:
                answer = []
                answer.append(str(pd.to_datetime(soup.find_all('td')[j*7].text, infer_datetime_format=True)))
                answer = answer + [i.span.text for i in soup.find_all('td')[j*7+1: j*7+7]] + [i]
                print(answer)

                connection = sqlite3.connect('MSC.db')
                cursor = connection.cursor()
                sql2 = f'cursor.execute("insert INTO  prices ({columnnames}) VALUES ({insertintotable})", {answer})'
                eval(sql2)
                connection.commit()
                connection.close()
                counter +=1
            except:pass        
        else:pass                

In [5]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
connection = sqlite3.connect("MSC.db")
cursor = connection.cursor()

sql = ('''Select distinct symbol, count(symbol) over(partition by symbol) as count from prices
order by count

       ''')
z =pd.read_sql_query(sql,connection)
z

In [ ]:
connection = sqlite3.connect("MSC.db")
cursor = connection.cursor()

sql = ('''Select * from sp500_companies

       ''')
z =pd.read_sql_query(sql,connection)
z

In [6]:
connection = sqlite3.connect("MSC.db")
cursor = connection.cursor()

sql = ('''Select sp500_companies.symbol, company from sp500_companies
          WHERE sp500_companies.Symbol NOT IN
        (Select symbol from  prices)

       ''')
z =pd.read_sql_query(sql,connection)
z

,Symbol,Company
0,BRK.B,Berkshire Hathaway Inc. Class B
1,BF.B,Brown-Forman Corporation Class B
2,FLIR,FLIR Systems Inc.


In [ ]:
# Flir 0 love 

In [7]:
connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()
sql = ('''
Select symbol from SP500_companies 
where symbol not in (select distinct symbol from prices where date == '2021-05-19 00:00:00')
and  symbol not in ('BRK.B', 'BF.B', 'FLIR')'''
    )
df = pd.read_sql_query(sql,connection)
df

,Symbol
0,AAPL
1,MSFT
2,RTX
3,GS
4,BA
...,...
440,UAA
441,UA
442,NWS
443,^GSPC


# Merge

In [ ]:
connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()

sql = ('''Select 
            main.date,
            top1,top2,top3,top4,top5,
            top6,top7,top8,top9,top10,
            top11,top12,top13,top14,top15,
            top16,top17,top18,top19,top20,
            top21,top22,top23,top24,top25,
            top26,top27,top28,top29,top30,
            
            DJI_Open,
            DJI_High,
            DJI_Low,
            DJI_Close,
            DJI_Adj_Close,
            
            SP500_Open,
            SP500_High,
            SP500_Low,
            SP500_Close,
            SP500_Adj_Close

            from main
            left join DJI on
            main.date = date(DJI.Date)
            left join SP500 on
            main.date = date(SP500.Date)
            
            where DJI_Open not NULL
            order by main.date

       ''')
df = pd.read_sql_query(sql,connection)
df.tail()

## Fix this 

In [12]:
import datetime

import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [99]:
#for symbol in df
no_financials = "'BRK.B', 'BF.B', 'FLIR'"


thisday = datetime.date.today().strftime('%Y%m%d')
connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()
sql = (f'''
Select symbol from SP500_companies 
where symbol not in (select distinct symbol from prices where date(date) == {thisday})
and  symbol not in ('BRK.B', 'BF.B', 'FLIR')'''
    )
df = pd.read_sql_query(sql,connection)

connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()
sql = ("Select * from prices order by date desc")
z = pd.read_sql_query(sql,connection)
connection.commit()
connection.close()

columnnames = "'date', 'open', 'high', 'low', 'close', 'adj_close','volume', 'symbol'"
insertintotable = '?,?,?,?,?,?,?,?'

In [130]:
#for symbol in df
no_financials = "'BRK.B', 'BF.B', 'FLIR'"


thisday = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
print(thisday)
connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()
sql = (f'''
Select symbol from SP500_companies 
where symbol not in (select distinct symbol from prices where date == "2021-05-19 00:00:00")
and  symbol not in ('BRK.B', 'BF.B', 'FLIR')'''
    )
df = pd.read_sql_query(sql,connection)

connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()
sql = ("Select * from prices order by date desc")
z = pd.read_sql_query(sql,connection)
connection.commit()
connection.close()

columnnames = "'date', 'open', 'high', 'low', 'close', 'adj_close','volume', 'symbol'"
insertintotable = '?,?,?,?,?,?,?,?'

for i in df.Symbol[:2]:
    r = requests.get(f'https://finance.yahoo.com/quote/{i}/history?p={i}')
    print(f'https://finance.yahoo.com/quote/{i}/history?p={i}')
    data = r.text
    soup = BeautifulSoup(data, features="lxml")
    
    for j in range (0, len(soup.find_all('td'))):
        try:
            if (type(pd.to_datetime(soup.find_all('td')[j].text,infer_datetime_format=True)) == pd._libs.tslibs.timestamps.Timestamp) == True and isinstance(locale.atof(soup.find_all('td')[j+1].text), (int, float, complex)) == True and pd.to_datetime(soup.find_all('td')[j].text,infer_datetime_format=True) > pd.to_datetime(z.date[z.symbol == i].iloc[0]):
                print(pd.to_datetime(soup.find_all('td')[j].text,infer_datetime_format=True))
                answer = []
                answer.append(str(pd.to_datetime(soup.find_all('td')[j].text,infer_datetime_format=True)))
                answer = answer + [i.span.text for i in soup.find_all('td')[j+1: j+7]] + [i]
                print(answer)

                connection = sqlite3.connect('MSC.db')
                cursor = connection.cursor()
                sql2 = f'cursor.execute("insert INTO  prices ({columnnames}) VALUES ({insertintotable})", {answer})'
                eval(sql2)
                connection.commit()
                connection.close()

            else: pass
        except:pass

2021-05-19
https://finance.yahoo.com/quote/AMZN/history?p=AMZN
2021-05-19 00:00:00
['2021-05-19 00:00:00', '3,195.00', '3,234.75', '3,184.00', '3,231.80', '3,231.80', '2,675,900', 'AMZN']
https://finance.yahoo.com/quote/GOOGL/history?p=GOOGL
2021-05-19 00:00:00
['2021-05-19 00:00:00', '2,228.43', '2,274.99', '2,225.44', '2,271.50', '2,271.50', '1,182,300', 'GOOGL']
